In [10]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import copy
import os
import time
from os import path as osp

import numpy as np
import sacred
import torch
import yaml
from sacred import Experiment
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from tracktor.config import get_output_dir
from tracktor.datasets.factory import Datasets
from tracktor.datasets.custom_wrapper import CustomSequence
from tracktor.frcnn_fpn import FRCNN_FPN
from tracktor.oracle_tracker import OracleTracker
from tracktor.reid.resnet import ReIDNetwork_resnet50
from tracktor.tracker import Tracker
from tracktor.utils import (evaluate_mot_accums, get_mot_accum,
                            interpolate_tracks, plot_sequence)

In [12]:
import cv2

In [13]:
frame_split = [0.0, 1.0]

In [19]:
obj_detects = []
obj_detect_model = "../../output/custom/model_epoch_30.model"
obj_detect = FRCNN_FPN(num_classes=31)
obj_detect.load_state_dict(torch.load(obj_detect_model, map_location=lambda storage, loc: storage))
obj_detects.append(obj_detect)
obj_detect.eval()
obj_detect.cuda();

In [20]:
tracker_conf = {
  "detection_person_thresh": 0.4,
  "regression_person_thresh": 0.1,
  "detection_nms_thresh": 0.1,
  "regression_nms_thresh": 0.6,
  "motion_model" : {
    "enabled": True,
    "n_steps": 5,
    "center_only": True
  },
  "public_detections": False,
  "do_align": True,
  "warp_mode": "MOTION_EUCLIDEAN",
  "number_of_iterations": 100,
  "termination_eps": 0.00001,
  "do_reid": False,
  "inactive_patience": 50,
  "max_features_num": 10,
  "reid_sim_threshold": 2.0,
  "reid_iou_threshold": 0.2
}
tracker = Tracker(obj_detect, [], tracker_conf)
tracker.reset()
num_frames = 0

In [21]:
name = "strandvejen_faxe"
result_dir = "../../output/custom/results"
seq = CustomSequence(name, result_dir)
data_loader = DataLoader(seq, batch_size=1, shuffle=False)

In [22]:
results = seq.load_results()
for i, frame in enumerate(tqdm(data_loader, leave=True, position=0)):
    if len(seq) * frame_split[0] <= i <= len(seq) * frame_split[1]:
        with torch.no_grad():
            tracker.step(frame, -1)
        num_frames += 1
        if num_frames == 100:
            break

results = tracker.get_results()
results = interpolate_tracks(results)
seq.write_results(results)

----------------
[tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 416.6459,  882.2505,  688.3312,  925.3332],
        [ 941.8119,  909.5068, 1009.6115,  949.0710],
        [ 541.4883,  666.9281, 1012.5342, 1019.2194]], device='cuda:0')
tensor([[ 421.8802,  891.7049,  702.6158,  939.6843],
        [ 931.7493,  911.4659, 1015.9495,  951.4286],
        [ 514.1317,  662.5268, 1017.0929, 1003.1561]], device='cuda:0')
----------------
----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 510.1155,  657.0143, 1013.5175,  996.9918],
        [ 405.3608,  898.9044,  717.8181,  949.1522],
        [ 980.4015,  930.9418, 1026.0731,  957.6268]], device='cuda:0')
tensor([[ 502.4328,  650.4576, 1004.6066, 1005.1447],
        [ 418.3500,  909.0012,  729.5765,  962.6600],
        [ 971.5266,  929.5036, 1024.0000,  957.9734]], device='cuda:0')
----------------
----------------
[tensor(16, device=

----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 533.2437,  664.4347, 1028.0172,  995.6929],
        [ 685.7068,  737.9131,  853.9713,  795.3354],
        [ 850.3441,  934.9773, 1016.9760,  988.4087]], device='cuda:0')
tensor([[ 525.2598,  657.6127, 1018.3062, 1005.1789],
        [ 681.8764,  737.5263,  861.4153,  800.7177],
        [ 875.7436,  938.5531, 1020.6920,  984.0258]], device='cuda:0')
----------------
----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 527.2441,  658.7014, 1021.5906, 1004.4161],
        [ 692.8672,  743.6604,  872.6419,  806.1781],
        [ 874.5148,  959.5771, 1027.1322, 1020.5005]], device='cuda:0')
tensor([[ 531.3080,  649.4703, 1013.3730, 1014.7356],
        [ 686.9540,  746.8600,  882.9370,  808.5696],
        [ 887.7321,  963.2819, 1024.0000, 1021.6400]], device='cuda:0')
----------------
----------------
[tensor(16, device=

----------------
[tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 534.1693,  663.2681, 1008.2443, 1023.7555],
        [  -7.9678,  845.1373,  128.3844,  968.3011],
        [ 364.4585,  620.8271,  502.7103,  681.5869],
        [ 976.8853,  977.7060, 1025.0735, 1018.2678]], device='cuda:0')
tensor([[ 510.5605,  660.8113, 1023.2352, 1020.9211],
        [   0.0000,  851.2462,  130.7402,  965.0930],
        [ 363.4979,  621.1946,  509.1521,  686.6022],
        [ 967.8509,  972.3342, 1024.0000, 1024.0000]], device='cuda:0')
----------------
----------------
[tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 508.7849,  662.8576, 1021.6846, 1022.6467],
        [  -7.2123,  854.8044,  123.5992,  968.5694],
        [ 362.5853,  619.6973,  508.2803,  685.0139]], device='cuda:0')
tensor([[ 502.8173,  645.5831, 1006.4548, 1019.3508],
        [   0.0000,  865.4911,  109.3024, 

----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 267.9540,  709.4934,  549.9835,  779.8707],
        [ 486.4639,  668.2778,  999.4777, 1012.7206]], device='cuda:0')
tensor([[ 266.8513,  710.8126,  551.3284,  782.4539],
        [ 450.6924,  669.1569,  996.5046, 1011.5201]], device='cuda:0')
----------------
----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(10, device='cuda:0')]
tensor([[ 269.7739,  715.5182,  554.2061,  787.3377],
        [ 445.1063,  669.1793,  990.7039, 1011.8843],
        [ 397.6220,  744.7493,  421.6400,  773.7836],
        [  18.5632,  676.3281,  339.6567,  789.2656],
        [ 477.3013,  770.2254,  552.7261,  789.1147]], device='cuda:0')
tensor([[ 257.8799,  713.5064,  559.3373,  789.7986],
        [ 427.9860,  657.6179,  950.0978, 1019.1851],
        [ 396.2325,  749.4958,  420.8167,  776.4529],
        [   4.4877,  678.9678,  342.3806, 

----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ 140.0011,  760.4201,  413.0941,  896.7914],
        [ -13.9416,  743.6870,  913.5371, 1030.3015],
        [  -2.2654,  697.6433,  317.0749,  821.6299],
        [ 386.0260,  819.3279,  411.8693,  887.2341],
        [ 142.0738,  803.2395,  562.6854,  883.8533],
        [ 291.9662,  738.8625,  463.1087,  764.8337],
        [ 775.5498,  812.7535,  892.2296,  850.6005],
        [ 494.1904,  891.0067,  514.3387,  939.3187],
        [ 359.3637,  866.3661,  482.2547,  887.4410],
        [ 856.4122,  611.8207,  914.8514,  779.4105],
        [ 490.6755,  758.4856,  570.7836,  778.7810],
        [ 9

----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(10, device='cuda:0'), tensor(10, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[ -21.1185,  828.3916,  356.2126, 1024.2799],
        [ -22.4656,  802.5040,  347.2662, 1017.6012],
        [  -1.7980,  715.5223,  299.2701,  804.9924],
        [ -23.0194,  812.3876,  334.7389, 1018.3435],
        [ 244.4400,  786.0856,  462.5209,  826.9598],
        [ 490.4602,  726.0569,  565.5266,  758.4504],
        [ 352.7907,  911.1936,  380.5673, 1006.0496],
        [ 942.4684,  947.4172, 1000.4467, 1004.7479],
        [ 462.5219,  904.6708,  702.2938, 1016.8170],
        [ 494.8644,  816.9634,  621.9802,  849.3533],
        [ 492.9846,  880.5151,  510.6841,  910.2336]], device='cuda:0')
tensor([[  18.3352,  812.1318,  369.9710, 1008.3244

----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[  22.7545,  893.4729,  505.0657, 1018.9702],
        [  22.8120,  893.0875,  506.5321, 1019.2979],
        [  21.3367,  893.1865,  504.8912, 1018.8621],
        [ 116.9748,  888.6859,  456.5445,  981.5180],
        [ 506.8780,  761.6014,  616.2578,  822.8184],
        [  19.9730,  965.6360,   95.2294, 1009.9190]], device='cuda:0')
tensor([[  32.7783,  906.3138,  466.4518, 1009.1780],
        [  33.1618,  906.1447,  466.0274, 1009.2398],
        [  33.1493,  906.4509,  466.3980, 1009.1580],
        [  65.2260,  899.1194,  463.1459, 1008.1794],
        [ 505.0200,  765.9807,  626.3822,  830.5238],
        [  22.2846,  957.9054,  129.4055, 1012.5955]], device='cuda:0')
----------------
----------------
[tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), tensor(16, device='cuda:0'), te

----------------
[tensor(16, device='cuda:0'), tensor(14, device='cuda:0')]
tensor([[ 566.7432,  961.2831,  793.7826, 1041.5244],
        [ 551.2344,  552.7646,  571.5518,  618.8074]], device='cuda:0')
tensor([[ 602.7836,  958.9356,  790.9177, 1024.0000],
        [ 551.2246,  554.0728,  571.8289,  624.5018]], device='cuda:0')
----------------
----------------
[tensor(14, device='cuda:0'), tensor(14, device='cuda:0')]
tensor([[ 553.3203,  552.7318,  573.8217,  623.1910],
        [ 688.3276,  973.0647,  788.3639, 1013.6774]], device='cuda:0')
tensor([[ 553.5308,  552.0753,  574.2350,  622.3190],
        [ 687.7006,  970.1066,  782.4729, 1011.0495]], device='cuda:0')
----------------
----------------
[tensor(14, device='cuda:0'), tensor(16, device='cuda:0')]
tensor([[556.9557, 551.0953, 577.7750, 621.3050],
        [276.9500, 723.2350, 460.9303, 962.5795]], device='cuda:0')
tensor([[ 556.2322,  550.9661,  577.3635,  627.7027],
        [ 257.9530,  748.2529,  453.0458, 1006.6282]], device=

In [9]:
results[0]

{0: array([4.14529205e+02, 8.82629517e+02, 6.86153687e+02, 9.26093445e+02,
        1.60000000e+01, 1.16398849e-03]),
 1: array([4.21880157e+02, 8.91704895e+02, 7.02615845e+02, 9.39684265e+02,
        1.60000000e+01, 1.16398849e-03]),
 2: array([4.25132050e+02, 9.03803284e+02, 7.14694214e+02, 9.51029663e+02,
        1.60000000e+01, 1.16398849e-03]),
 3: array([4.14073120e+02, 9.16509888e+02, 7.39567688e+02, 9.62960815e+02,
        1.60000000e+01, 1.16398849e-03])}